In [23]:
load ("C:/Users/marci/Documentos/GitHub/ECO224/Labs/data/wage2015_subsample_inference.Rdata")
attach(data)

dim(data)

[1] 5150   20

In [81]:
library(xtable)

Z <- data[which(colnames(data) %in% c("lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"))]

data_somecoll <- data[data$scl==1,]
Z_somecoll <- data_somecoll[which(colnames(data) %in% c("lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"))]

data_college <- data[data$clg==1,]
Z_coll <- data_college[which(colnames(data) %in% c("lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"))]

table <- matrix(0, 12, 3)
table[1:12,1] <- as.numeric(lapply(Z,mean))    
table[1:12,2] <- as.numeric(lapply(Z_somecoll,mean))
table[1:12,3] <- as.numeric(lapply(Z_coll,mean))
rownames(table) <- c("Log Wage","Sex","Less then High School","High School Graduate","Some College","Gollage Graduate","Advanced Degree", "Northeast","Midwest","South","West","Experience")
colnames(table) <- c("All","Some College","College")   
tab <- xtable(table, digits =4)
tab

base <- rbind(Z_somecoll,Z_coll)

base1<- rbind(data_somecoll,data_college)

base2<- base1[["wage","lwage","sex","shs","hsg","ad","mw","so","we","ne","exp1","exp2","exp3","exp4","occ","occ2","ind","ind2",]]

education <-matrix(c(1),nrow=3068,ncol=1)
colnames(education) <- c("Education")
education

base2<- rbind(base2[],education)



,All,Some College,College
Log Wage,2.97078670,2.8473351,3.1336703
Sex,0.44446602,0.4539106,0.4859413
Less then High School,0.02330097,0.0000000,0.0000000
High School Graduate,0.24388350,0.0000000,0.0000000
Some College,0.27805825,1.0000000,0.0000000
Gollage Graduate,0.31766990,0.0000000,1.0000000
Advanced Degree,0.13708738,0.0000000,0.0000000
Northeast,0.25961165,0.3058659,0.2310513
Midwest,0.29650485,0.2821229,0.2891198
South,0.21611650,0.2255587,0.2182152


ERROR: Error in col[[i, exact = exact]]: subíndice fuera de  los límites


In [ ]:
library(sandwich)
nocontrol.fit <- lm(lwage ~ education )
nocontrol.est <- summary(nocontrol.fit)$coef["education",1]
HCV.coefs <- vcovHC(nocontrol.fit, type = 'HC');
nocontrol.se <- sqrt(diag(HCV.coefs))[2] # Estimated standard errors

cat("The estimated education coefficient is", nocontrol.est, " and the corresponding robust standar error is", nocontrol.se)


In [ ]:
#OLS regression with controls
flex <- lwage ~ education + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)

control.fit <- lm(flex, data=data)
control.est <- summary(control.fit)$coef[2,1]

summary(control.fit)

cat("Coefficient for OLS with controls", control.est)

HCV.coefs <- vcovHC(control.fit, type = 'HC');
control.se <- sqrt(diag(HCV.coefs))[2] #Estimated stantard errors


In [ ]:
#Partialling-Out using Ols

#models
flex.y <- lwage ~ (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we) # model for Y
flex.d <- educaction ~ (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we) # model for D

#partiallling-out the linear effect of W from Y
t.Y <- lm(flex.y, data=data)$res
# partialling-out the linear effect of W from D
t.D <- lm(flex.d, data=data)$res

# regression of Y on D after partialling-out the effect of W
partial.fit <- lm(t.Y~t.D)
partial.est <- summary(partial.fit)$coef[2,1]

cat("Coefficient for D via partialling-out", partial.est)

# standard error
HCV.coefs <- vcovHC(partial.fit, type = 'HC')
partial.se <- sqrt(diag(HCV.coefs))[2]

# confidence interval
confint(partial.fit)[2,]
